In [2]:
!pip install langchain_community
!pip install pypdfium2
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.5 MB/s eta 0:00:00


In [5]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 47.5 MB/s eta 0:00:00


In [16]:
!pip install rank_bm25

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def preprocessing_pdf(text):
    """특정 페이지 전체 및 불필요한 텍스트 제거"""
    text = re.sub(r'KOSHA GUIDE', '', text)
    text = re.sub(r'^C - .+$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^<그림\s*\d+\s*>$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*-\s*\d+\s*-\s*$', '', text, flags=re.MULTILINE)
    return text.strip()

# PDF 파일들이 있는 폴더 경로 설정
pdf_dir = "/content/drive/MyDrive/PDF"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
text_splitter = SemanticChunker(embeddings)

all_chunks = []

# tqdm을 사용하여 PDF 파일 처리 진행 상황 추적
for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    print(f"\nProcessing: {pdf_file}")
    loader = PyPDFium2Loader(pdf_file)
    documents = loader.load()
    # 3페이지(인덱스 2)부터 전처리한 텍스트 추출
    page_contents = []
    for i, doc in enumerate(documents[2:], start=3):
        processed_text = preprocessing_pdf(doc.page_content)
        page_contents.append(processed_text)
        print(f"  Processed page {i}")
    # 문서별로 페이지 텍스트들을 합치기
    doc_text = "\n".join(page_contents)
    # 문서별로 청크 생성
    chunks = text_splitter.split_text(doc_text)
    print(f"  Generated {len(chunks)} chunks")
    all_chunks.extend(chunks)

vectorstore = FAISS.from_texts(all_chunks, embeddings)
vectorstore.save_local("faiss_index")
print("FAISS index saved successfully.")


In [7]:
vectorstore

In [10]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [17]:
# Retriever 정의
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document

docs = [Document(page_content=doc) for doc in all_chunks]
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer)


In [21]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.0, 1.0]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)

In [22]:
# 사용자 질의 예시
query = "건축물' 공사 중 철근콘크리트공사' 작업에서  '타설작업 작업프로세스 진행중 부딪힘 발생 했습니다 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?"

# ensemble_retriever를 이용해 관련 문서 검색 (두 리트리버의 결과를 결합하여 가중치에 따라 정렬)
retrieved_docs = ensemble_retriever.get_relevant_documents(query)

# 검색된 문서 출력
print("검색된 문서들:")
for idx, doc in enumerate(retrieved_docs):
    print(f"\n문서 {idx+1}:")
    print(doc.page_content)

검색된 문서들:

문서 1:
(4) 굴착장비의 효율성과 구조물의 안정성을 고려하여 패널 분할계획을 수립하여야
한다. (5) 트레미관은 지중에 콘크리트를 타설 하기 위한 것으로 지상에서 관을 통하여
콘크리트를 자유 낙하시켜 타설 함으로 관 접합부의 막힘으로 인한 터짐을 방지
하도록 콘크리트 재료, 타설 관리를 철저히 하여 비산 및 낙하․비래 사고를
방지하여야 한다. (6) 지하연속벽의 시공오차 발생에 대한 처리방법은 마감 공간 등을 고려하여 설계
단계에서부터 오차기준을 정하여 시공에 반영하여야 한다. (7) 철근 배근작업 시 고려사항은 아래와 같다
(가) 인양 시 변형발생 방지를 위해 “X”자 형태로 보강한다. (나) 트레미관 설치를 위한 공간을 확보한다.

문서 2:
(3) 콘크리트 타설 후 급격한 온도변화 방지를 위해 보일러 가동이 중단되지
않도록 해야 한다. 5.7 추진코
5.7.1 작업개요
부 모우멘트와 처짐방지를 위하여 선단부에 세그멘트(Segmen) 길이의 60~ 70%를
조립하여 연결하는 철골 트러스이며 선단부에 유압잭을 설치하여 압출선단의
처짐량을 조정하여야 한다
<그림 9> 추진코
5.7.2 작업시 준수사항
(1) 집중적인 프리스트레스의 작용단면이 발생하지 않도록 분할배치 시공하여야
한다. (2) 추진코와 본체 구조물 연결부는 충분한 전단력을 발휘할 수 있는 체결방식
이어야 한다. (3) 추진코의 제작 조립거치 시 추진코의 하단과 본체 구조물의 하단이 정확히 동일
평면에 있어야 하고, 거더의 추진에 지장이 없도록 충분한 정밀도를 가져야 한다. (4) 추진코 부재 반입 시 안전설비 부착 유무를 확인하여야 한다. (5) 추진코의 전도방지를 위한 지주 설치 등 전도방지 조치를 실시하여야 한다. (6) 크레인으로 인양 및 조립·해체작업 시 전도방지를 위한 다음과 같은 조치를
하여야 한다. (가) 지반다짐 및 평탄 작업
(나) 아웃트리거 하부에 받침목 설치
(다) 필요시 콘크리트 타설 및 철판 사용
(7) 추진코가 안전하게 위치할 때까지 크레인으로 고정하